# Prise en main de `beautifulsoup` (suite)

<div class="alert alert-block alert-info">
<bObjectif p:</b>Ce notebook permet de découvrir les commandes de base de <code>beautifulsoup</code>, en s'appuyant sur le site <a href="https://quotes.toscrape.com/"> https://quotes.toscrape.com/</a>. </div>

🛟 Pour la suite, éditez de nouveau https://quotes.toscrape.com/ dans _Chrome_ et affichez-y les _outils de développement_. Affichez à l'écran en parallèle l'instance de Chrome ouverte et ce notebook.

## Chargement des modules python

Pour ce TP, 2 modules sont nécessaires :

In [1]:
import requests
import bs4      # <= beautifulsoup

## Navigation vers la page suivante

Partant du portail de https://quotes.toscrape.com/, on peut naviguer vers la page suivante de citation par un simple clic sur le lien `Next` (mis en forme avec un style de bouton). 

Observez le code source du bouton `Next`. On peut donc récupérer l'élément associé et l'URL du lien avec :

In [3]:
# Charge la page 
url = "http://quotes.toscrape.com/"
response = requests.get(url)
source = response.text 
# Parse la page
html = bs4.BeautifulSoup(source, 'html.parser')
# Récupère le lien du bouton Next
next = html.select("li.next a")[0] # un seul élément
lien_next = next.get("href")
print(lien_next)

/page/2/


## Gestion d'URL

L'URL ciblée est une URL relative à la racine du site Web. Pour fabriquer l'URL complète, il faut _joindre_ le nom de domaine (déjà contenu dans `url`) et la cible. Pour ce faire, on peut utiliser le module `urllib.parse` et sa fonction `urljoin` :  

In [4]:
import urllib.parse

url_next = urllib.parse.urljoin(url, lien_next)
print(url_next)

http://quotes.toscrape.com/page/2/


Dans le navigateur, rendez-vous sur http://quotes.toscrape.com/page/2/, ce qui d'un point de vue code renvient à : 

In [6]:
# Charge la 2ème page 
response = requests.get(url_next)
source = response.text 
# Parse la page
html = bs4.BeautifulSoup(source, 'html.parser')

## Navigation vers la 3ème page

Observez le code source de cette seconde page. Le bouton `Next` est toujours présent, avec le même sélecteur CSS (`li.next a`). On peut donc le récupérer et déduire l'url de la page n°3.

In [12]:
# Récupère le lien du bouton Next pour aller de la page 2 à la page 3
next = html.select("li.next a")[0] # un seul élément
print(html.select("li.next a"))
lien_next = next.get("href")
print(lien_next)
url_next = urllib.parse.urljoin(url, lien_next)
print(url_next)

[<a href="/page/3/">Next <span aria-hidden="true">→</span></a>]
/page/3/
http://quotes.toscrape.com/page/3/


## Détection de la dernière page

On peut ainsi procéder de la sorte jusqu'à ce qu'on atteigne la dernière page de citation. Rendez-vous sur cette dernière page (essayez par exemple https://quotes.toscrape.com/page/30/). 

Pour détecter la fin des pages de citation, on ne pourra pas utiliser `requests` (avec une réponse autre que le code 200) car le site renvoie bien un code source. Par contre, sur cette page, le bouton `Next` n'existe plus, signe qu'on a atteint le nombre de citation limite. On peut le détecter avec :

In [13]:
# Charge la 30ème page 
response = requests.get("https://quotes.toscrape.com/page/30/")
source = response.text 
# Parse la page
html = bs4.BeautifulSoup(source, 'html.parser')
try:
    # Récupère le lien du bouton Next pour aller de la page 2 à la page 3
    next = html.select("li.next a")[0] # un seul élément
except:
    print("Le bouton Next n'existe pas")

Le bouton Next n'existe pas


## Synthèse : navigation de pages en pages

Finalement, une navigation de pages en pages du site `https://quotes.toscrape.com/` peut être réalisée à l'aide d'une boucle de la forme : 

In [14]:
nom_site = "http://quotes.toscrape.com/"

a_charger = nom_site

fin_navigation = False
while not fin_navigation:
    # Charge la 30ème page 
    print(f"Chargement de {a_charger}")
    response = requests.get(a_charger)
    source = response.text 
    
    # Parse la page
    html = bs4.BeautifulSoup(source, 'html.parser')
    try:
        # Récupère le lien du bouton Next pour aller de la page 2 à la page 3
        next = html.select("li.next a")[0] # un seul élément
        lien_next = next.get("href") # lien de la prochaine page
        a_charger = urllib.parse.urljoin(url, lien_next) # URL cible pour l'itération suivante
    except:
        fin_navigation = True
        print("Le bouton Next n'existe pas")

Chargement de http://quotes.toscrape.com/
Chargement de http://quotes.toscrape.com/page/2/
Chargement de http://quotes.toscrape.com/page/3/
Chargement de http://quotes.toscrape.com/page/4/
Chargement de http://quotes.toscrape.com/page/5/
Chargement de http://quotes.toscrape.com/page/6/
Chargement de http://quotes.toscrape.com/page/7/
Chargement de http://quotes.toscrape.com/page/8/
Chargement de http://quotes.toscrape.com/page/9/
Chargement de http://quotes.toscrape.com/page/10/
Le bouton Next n'existe pas


Chaque page chargée peut bien sur être traitée avec beautifulsoup pour en extraire des données...